In [ ]:
# default_exp core

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
!pip install -qU nbdev fastcore

In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

# Core
> Core tools


## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread

In [ ]:
%%capture
#hide

import yaml
with open("facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
import os
from typing import *

import gspread
from gspread.models import *
import pandas as pd
import numpy as np

APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")
SHEET = os.environ.get("SHEET", "Followers API")
SH = gspread.service_account(filename="creds.json").open(SHEET)

## Google Sheets

### Insert new followers to a google sheet 

In [ ]:
#export
def get_df(worksheet: str) -> pd.DataFrame:
    try:
        wsh = SH.worksheet(worksheet)
        df = pd.DataFrame(wsh.get_all_records()).replace("", 0)
        index = df.columns[0]
        df.set_index(df[index], inplace=True)
        return df.drop(columns=index)
    except WorksheetNotFound as e:
        return pd.DataFrame()

In [ ]:
assert "AR" in get_df("History").index

In [ ]:
#export
def get_worksheet(name: str, loc: int=6) -> Worksheet:
    worksheets = [w.title for w in SH.worksheets()]
    if name in worksheets:
        return SH.worksheet(name)
    else:
        return SH.add_worksheet(name, 1, 1, loc)

In [ ]:
#export
def write_df(df: pd.DataFrame, worksheet: str, loc: str = "A1", columns:bool = True, worksheet_loc: int=6):
    """Write a dataframe to a sheet without changing it"""
    worksheet = get_worksheet(worksheet, worksheet_loc)
    df = df.copy()
    df.insert(0, df.index.name, df.index)
    df.replace([0, np.inf, np.nan, float("-inf")], "", inplace=True)
    values = [df.columns.to_list()] + df.values.tolist() if columns else df.values.tolist()
    worksheet.update(loc, values, raw=False)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
